In [ ]:
## chicago 
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, BooleanType, DateType, IntegerType, FloatType

df = spark.read.option("header", "true").csv("downloads/chicago_data.csv")

df1 = df.drop('Case Number', 'Block', 'IUCR', 'Ward', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'District', 'FBI Code', 'Community Area', 'Updated On', 'Description', 'Location', ' Updated On', 'Latitude', 'Longitude')


df2 = df1.withColumn("Date", col('Date').cast(DateType())).withColumn('ID', col('ID').cast(IntegerType())).withColumn('X Coordinate', col('X Coordinate').cast(FloatType())).withColumn('Y Coordinate', col('Y Coordinate').cast(FloatType())).withColumn('Year', col('Year').cast(IntegerType()))

df3 = df2.filter(df.Year == 2018)

df4 = df3.filter(df3.Year.isNotNull() & df3.Date.isNotNull() & df3.ID.isNotNull())

chicago_cleaned = df4.filter(col('Primary Type').isNotNull()).filter(col('X Coordinate').isNotNull()).filter(col("Y Coordinate").isNotNull())

chicagocomplaint = chicago_cleaned.groupBy("Date").count()

## la 

dfa = spark.read.option("header", "true").csv("downloads/la_data.csv")

df1a = dfa.drop('Report Type', 'Time', 'Area ID', 'Area Name', 'Reporting District', 'Age', 'Sex Code', 'Descent Code', 'Charge Group Code', 'Charge Group Description', 'Arrest Type Code', 'Charge', 'Disposition Description', 'Address', 'Cross Street', 'Location', 'Booking Date', 'Booking Time', 'Booking Location', 'Booking Location Code')


df2a = df1a.withColumnRenamed("Arrest Date","Date").withColumnRenamed("Report ID", "ID").withColumnRenamed("Charge Description", "Primary Type").withColumnRenamed("LAT", "Latitude").withColumnRenamed("LON", "Longitude")


df3a = df2a.withColumn("Date", col('Date').cast(DateType())).withColumn('ID', col('ID').cast(IntegerType())).withColumn('Latitude', col('Latitude').cast(FloatType())).withColumn('Longitude', col('Longitude').cast(FloatType()))

df4a = df3a.filter(df3a.Date > '2017-12-31')
df4b = df4a.filter('2019-01-01' > df4a.Date)


df5a = df4b.filter(col('Primary Type').isNotNull())

la_cleaned = df5a.filter(df5a.ID.isNotNull() & df5a.Date.isNotNull() & df5a.Latitude.isNotNull() & df5a.Longitude.isNotNull())

lacomplaint = la_cleaned.groupBy("Date").count()

## ny

dfn = spark.read.option("header", "true").csv("downloads/ny_data.csv")

df1n = dfn.drop('PD_DESC', 'PD_CD', 'KY_CD', 'LAW_CODE', 'LAW_CAT_CD', 'ARREST_BORO', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'X_COORD_CD', 'Y_COORD_CD', 'Lon_Lat')

df2n = df1n.withColumnRenamed("ARREST_KEY", "ID").withColumnRenamed("ARREST_DATE", "Date").withColumnRenamed("OFNS_DESC", "Primary Type")

df3n = df2n.withColumn("Date", col('Date').cast(DateType())).withColumn('ID', col('ID').cast(IntegerType())).withColumn('Latitude', col('Latitude').cast(FloatType())).withColumn('Longitude', col('Longitude').cast(FloatType()))

df4n = df3n.filter(df3n.Date > '2017-12-31')
df4m = df4n.filter('2019-01-01' > df4n.Date)


df5n = df4m.filter(col('Primary Type').isNotNull())

ny_cleaned = df5n.filter(df5n.ID.isNotNull() & df5n.Date.isNotNull() & df5n.Latitude.isNotNull() & df5n.Longitude.isNotNull()) 

nycomplaint = ny_cleaned.groupBy("Date").count()

# chicago
dfcw = spark.read.option("header", "true").csv("downloads/chicago_weather.csv")

dfcw1 = dfcw.withColumnRenamed("time", "Date")

dfcw2 = dfcw1.drop('temperature_2m_max (°F)', 'temperature_2m_min (°F)', 'rain_sum (inch)', 'precipitation_sum (inch)', 'snowfall_sum (cm)')


dfcw3 = dfcw2.withColumn("Date", col('Date').cast(DateType())).withColumn('apparent_temperature_min (°F)', col('apparent_temperature_min (°F)').cast(FloatType())).withColumn('apparent_temperature_max (°F)', col('apparent_temperature_max (°F)').cast(FloatType()))


# new york
dfnw = spark.read.option("header", "true").csv("downloads/ny_weather.csv")

dfnw1 = dfnw.withColumnRenamed("time", "Date")

dfnw2 = dfnw1.drop('temperature_2m_max (°F)', 'temperature_2m_min (°F)', 'rain_sum (inch)', 'precipitation_sum (inch)', 'snowfall_sum (cm)')

dfnw3 = dfnw2.withColumn("Date", col('Date').cast(DateType())).withColumn('apparent_temperature_min (°F)', col('apparent_temperature_min (°F)').cast(FloatType())).withColumn('apparent_temperature_max (°F)', col('apparent_temperature_max (°F)').cast(FloatType()))


# los angeles 
dflw = spark.read.option("header", "true").csv("downloads/la_weather.csv")

dflw1 = dflw.withColumnRenamed("time", "Date")

dflw2 = dflw1.drop('temperature_2m_max (°F)', 'temperature_2m_min (°F)', 'rain_sum (inch)', 'precipitation_sum (inch)', 'snowfall_sum (cm)')


dflw3 = dflw2.withColumn("Date", col('Date').cast(DateType())).withColumn('apparent_temperature_min (°F)', col('apparent_temperature_min (°F)').cast(FloatType())).withColumn('apparent_temperature_max (°F)', col('apparent_temperature_max (°F)').cast(FloatType()))



# average temps

from pyspark.sql.functions import col

chicagoweather = dfcw3.withColumn("Average Apparent Temperature", (col('apparent_temperature_max (°F)') + col('apparent_temperature_min (°F)'))/2)

chicago_weather = chicagoweather.drop('apparent_temperature_min (°F)', 'apparent_temperature_max (°F)')

nyweather = dfnw3.withColumn("Average Apparent Temperature", (col('apparent_temperature_max (°F)') + col('apparent_temperature_min (°F)'))/2)

ny_weather = nyweather.drop('apparent_temperature_min (°F)', 'apparent_temperature_max (°F)')

laweather = dflw3.withColumn("Average Apparent Temperature", (col('apparent_temperature_max (°F)') + col('apparent_temperature_min (°F)'))/2)

la_weather = laweather.drop('apparent_temperature_min (°F)', 'apparent_temperature_max (°F)')

# find monthly average temperature


ny_avg_temp_per_month = avg_ny.groupBy("DateMonth").avg("Average Apparent Temperature")

c_avg_temp_per_month = avg_chicago.groupBy("DateMonth").avg("Average Apparent Temperature")

la_avg_temp_per_month = avg_la.groupBy("DateMonth").avg("Average Apparent Temperature")



## FINAL DFS FOR MODELING 

newyork = ny_avgs.alias("a").join(avg_ny.alias("b"), col('a.DateMonth') == col('b.DateMonth')).drop(ny_avgs.DateMonth)
newyork = newyork.drop(newyork.DateMonth)

ny_df = newyork.withColumnRenamed("avg(count)", 'AverageCrimesPerMonth').withColumnRenamed("avg(Average Apparent Temperature)", "AverageTemperaturePerMonth").withColumnRenamed("count", 'NumberOfCrimesPerDay').withColumnRenamed("Average Apparent Temperature", 'DailyTemperature')

ny_df = ny_df.select('Date', 'DailyTemperature', 'NumberOfCrimesPerDay', 'AverageCrimesPerMonth', 'AverageTemperaturePerMonth', "Location")

chicago = c_avgs.alias("a").join(avg_chicago.alias("b"), col('a.DateMonth') == col('b.DateMonth')).drop(c_avgs.DateMonth)

chicago = chicago.drop(chicago.DateMonth)

chicago_df = chicago.withColumnRenamed("avg(count)", 'AverageCrimesPerMonth').withColumnRenamed("avg(Average Apparent Temperature)", "AverageTemperaturePerMonth").withColumnRenamed("count", 'NumberOfCrimesPerDay').withColumnRenamed("Average Apparent Temperature", 'DailyTemperature')

chicago_df = chicago_df.select('Date', 'DailyTemperature', 'NumberOfCrimesPerDay', 'AverageCrimesPerMonth', 'AverageTemperaturePerMonth', "Location")

la = la_avgs.alias("a").join(avg_la.alias("b"), col('a.DateMonth') == col('b.DateMonth')).drop(la_avgs.DateMonth)
la = la.drop(la.DateMonth)

la_df = la.withColumnRenamed("avg(count)", 'AverageCrimesPerMonth').withColumnRenamed("avg(Average Apparent Temperature)", "AverageTemperaturePerMonth").withColumnRenamed("count", 'NumberOfCrimesPerDay').withColumnRenamed("Average Apparent Temperature", 'DailyTemperature')

la_df = la_df.select('Date', 'DailyTemperature', 'NumberOfCrimesPerDay', 'AverageCrimesPerMonth', 'AverageTemperaturePerMonth', "Location")

## concatenated FINAL DATA 

combine1 = ny_df.unionByName(chicago_df)

final = combine1.unionByName(la_df)


